# Bag of Words in R with `text2vec`

Unlike `tidytext`, `text2vec` is more oriented towards document-level transformations.  But like `tidytext`, it's designed to provide a few well-designed, narrow tools that you stick into your data processing pipeline.  (As opposed to: being a one-stop shop for all your NLP needs).  It also provides a nice pre-built tool for replicable, stateful tokens-to-matrix transformations.  Personally, I like `text2vec` more than `tidytext`, but the practical differences are pretty minor.

In [1]:
# Requirements
# install.packages("dplyr")      # if you don't know what dplyr is I can't help you
# install.packages("magrittr")   # pipes!
# install.packages("naivebayes") # Naive Bayes implementation
# install.packages("SnowballC")  # stemming
# install.packages("text2vec")   # tokenization and document-term matrix creation
# install.packages("yardstick")  # model metrics; part of the tidymodels suite

In [2]:
library(dplyr)
library(magrittr)
library(naivebayes)
library(SnowballC)
library(text2vec)
library(yardstick)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


naivebayes 0.9.7 loaded

For binary classification, the first factor level is assumed to be the event.
Use the argument `event_level = "second"` to alter this as needed.



# TidyText

TidyText has a lot of pretty easy-to-use tools that integrate right into the broader Tidyverse ecosystem.  TidyText doesn't actually offer a whole lot of tools--mostly, just an `unnest_tokens()` function that converts a column of strings into the "one token per row" format.  But once everything is in that format, it's pretty easy to work with using existing Tidyverse tools and techniques.  E.g. filtering, groupby-aggregate, etc.; it's a format that's very friendly to _stateless_ and even parallelized operations over tokens.

In [3]:
# Load data
train <- read.csv("../../data/train.csv", stringsAsFactors = FALSE)
test <- read.csv("../../data/test.csv", stringsAsFactors = FALSE)
str(train)

'data.frame':	200000 obs. of  8 variables:
 $ review_id       : chr  "en_0964290" "en_0690095" "en_0311558" "en_0044972" ...
 $ product_id      : chr  "product_en_0740675" "product_en_0440378" "product_en_0399702" "product_en_0444063" ...
 $ reviewer_id     : chr  "reviewer_en_0342986" "reviewer_en_0133349" "reviewer_en_0152034" "reviewer_en_0656967" ...
 $ stars           : int  1 1 1 1 1 1 1 1 1 1 ...
 $ review_body     : chr  "Arrived broken. Manufacturer defect. Two of the legs of the base were not completely formed, so there was no wa"| __truncated__ "the cabinet dot were all detached from backing... got me" "I received my first order of this product and it was broke so I ordered it again. The second one was broke in m"| __truncated__ "This product is a piece of shit. Do not buy. Doesn't work, and then I try to call for customer support, it won'"| __truncated__ ...
 $ review_title    : chr  "I'll spend twice the amount of time boxing up the whole useless thing and send it back wit

Let's do some of the same cleanup we did earlie with `tidytext`:

In [4]:
tokens <- (
    train$review_body
    %>% tolower()
    %>% gsub("[^a-z]+", " ", .)
)

`text2vec` provides its own tokenizers (we'll use `word_tokenizer()`, but there are others; check the documentation), and tools for converting the tokenized text into an iterator format that most of the rest of its functions seem to expect:

In [5]:
tokens <- word_tokenizer(tokens)
head(tokens, 10)

[[1]]
  [1] "arrived"      "broken"       "manufacturer" "defect"       "two"         
  [6] "of"           "the"          "legs"         "of"           "the"         
 [11] "base"         "were"         "not"          "completely"   "formed"      
 [16] "so"           "there"        "was"          "no"           "way"         
 [21] "to"           "insert"       "the"          "casters"      "i"           
 [26] "unpackaged"   "the"          "entire"       "chair"        "and"         
 [31] "hardware"     "before"       "noticing"     "this"         "so"          
 [36] "i"            "ll"           "spend"        "twice"        "the"         
 [41] "amount"       "of"           "time"         "boxing"       "up"          
 [46] "the"          "whole"        "useless"      "thing"        "and"         
 [51] "send"         "it"           "back"         "with"         "a"           
 [56] "star"         "review"       "of"           "part"         "of"          
 [61] "a"            "chair"        "i"            "never"        "got"         
 [66] "to"           "sit"          "in"           "i"            "will"        
 [71] "go"           "so"           "far"          "as"           "to"          
 [76] "include"      "a"            "picture"      "of"           "what"        
 [81] "their"        "injection"    "molding"      "and"          "quality"     
 [86] "assurance"    "process"      "missed"       "though"       "i"           
 [91] "will"         "be"           "hesitant"     "to"           "buy"         
 [96] "again"        "it"           "makes"        "me"           "wonder"      
[101] "if"           "there"        "aren"         "t"            "missing"     
[106] "structures"   "and"          "supports"     "that"         "don"         
[111] "t"            "impede"       "the"          "assembly"     "process"     

[[2]]
 [1] "the"      "cabinet"  "dot"      "were"     "all"      "detached"
 [7] "from"     "backing"  "got"      "me"      

[[3]]
 [1] "i"        "received" "my"       "first"    "order"    "of"      
 [7] "this"     "product"  "and"      "it"       "was"      "broke"   
[13] "so"       "i"        "ordered"  "it"       "again"    "the"     
[19] "second"   "one"      "was"      "broke"    "in"       "more"    
[25] "places"   "than"     "the"      "first"    "i"        "can"     
[31] "t"        "blame"    "the"      "shipping" "process"  "as"      
[37] "it"       "s"        "shrink"   "wrapped"  "and"      "boxed"   

[[4]]
 [1] "this"     "product"  "is"       "a"        "piece"    "of"      
 [7] "shit"     "do"       "not"      "buy"      "doesn"    "t"       
[13] "work"     "and"      "then"     "i"        "try"      "to"      
[19] "call"     "for"      "customer" "support"  "it"       "won"     
[25] "t"        "take"     "my"       "number"   "fucking"  "rip"     
[31] "off"     

[[5]]
 [1] "went"    "through" "in"      "one"     "day"     "doesn"   "t"      
 [8] "fit"     "correct" "and"     "couldn"  "t"       "get"     "bubbles"
[15] "out"     "better"  "without"

[[6]]
 [1] "poor"       "quality"    "the"        "material"   "was"       
 [6] "fuzzy"      "from"       "day"        "one"        "it"        
[11] "got"        "discolored" "in"         "less"       "than"      
[16] "a"          "month"      "even"       "though"     "we"        
[21] "kept"       "it"         "outdoors"   "under"      "a"         
[26] "covered"    "porch"      "i"          "returned"   "this"      
[31] "item"      

[[7]]
 [1] "ordered"  "they"     "shipped"  "promised" "by"       "certain" 
 [7] "day"      "then"     "the"      "next"     "day"      "then"    
[13] "the"      "next"     "day"     

[[8]]
[1] "followed"   "directions" "did"        "not"        "work"      
[6] "as"         "advertised"

[[9]]
 [1] "there"     "is"        "a"         "terribly"  "done"      "band"     
 [7] "across"    "the"       "top"       "supposed"  "to"        "keep"     
[13] "the"       "shirt"     "o

If we want to apply stemming, we should do it now:

In [6]:
tokens <- lapply(tokens, wordStem)
head(tokens)

[[1]]
  [1] "arriv"      "broken"     "manufactur" "defect"     "two"       
  [6] "of"         "the"        "leg"        "of"         "the"       
 [11] "base"       "were"       "not"        "complet"    "form"      
 [16] "so"         "there"      "wa"         "no"         "wai"       
 [21] "to"         "insert"     "the"        "caster"     "i"         
 [26] "unpackag"   "the"        "entir"      "chair"      "and"       
 [31] "hardwar"    "befor"      "notic"      "thi"        "so"        
 [36] "i"          "ll"         "spend"      "twice"      "the"       
 [41] "amount"     "of"         "time"       "box"        "up"        
 [46] "the"        "whole"      "useless"    "thing"      "and"       
 [51] "send"       "it"         "back"       "with"       "a"         
 [56] "star"       "review"     "of"         "part"       "of"        
 [61] "a"          "chair"      "i"          "never"      "got"       
 [66] "to"         "sit"        "in"         "i"          "will"      
 [71] "go"         "so"         "far"        "a"          "to"        
 [76] "includ"     "a"          "pictur"     "of"         "what"      
 [81] "their"      "inject"     "mold"       "and"        "qualiti"   
 [86] "assur"      "process"    "miss"       "though"     "i"         
 [91] "will"       "be"         "hesit"      "to"         "bui"       
 [96] "again"      "it"         "make"       "me"         "wonder"    
[101] "if"         "there"      "aren"       "t"          "miss"      
[106] "structur"   "and"        "support"    "that"       "don"       
[111] "t"          "imped"      "the"        "assembli"   "process"   

[[2]]
 [1] "the"     "cabinet" "dot"     "were"    "all"     "detach"  "from"   
 [8] "back"    "got"     "me"     

[[3]]
 [1] "i"       "receiv"  "my"      "first"   "order"   "of"      "thi"    
 [8] "product" "and"     "it"      "wa"      "broke"   "so"      "i"      
[15] "order"   "it"      "again"   "the"     "second"  "on"      "wa"     
[22] "broke"   "in"      "more"    "place"   "than"    "the"     "first"  
[29] "i"       "can"     "t"       "blame"   "the"     "ship"    "process"
[36] "a"       "it"      ""        "shrink"  "wrap"    "and"     "box"    

[[4]]
 [1] "thi"     "product" "i"       "a"       "piec"    "of"      "shit"   
 [8] "do"      "not"     "bui"     "doesn"   "t"       "work"    "and"    
[15] "then"    "i"       "try"     "to"      "call"    "for"     "custom" 
[22] "support" "it"      "won"     "t"       "take"    "my"      "number" 
[29] "fuck"    "rip"     "off"    

[[5]]
 [1] "went"    "through" "in"      "on"      "dai"     "doesn"   "t"      
 [8] "fit"     "correct" "and"     "couldn"  "t"       "get"     "bubbl"  
[15] "out"     "better"  "without"

[[6]]
 [1] "poor"     "qualiti"  "the"      "materi"   "wa"       "fuzzi"   
 [7] "from"     "dai"      "on"       "it"       "got"      "discolor"
[13] "in"       "less"     "than"     "a"        "month"    "even"    
[19] "though"   "we"       "kept"     "it"       "outdoor"  "under"   
[25] "a"        "cover"    "porch"    "i"        "return"   "thi"     
[31] "item"

In [7]:
tokens <- itoken(tokens)
tokens

<itoken>
  Inherits from: <CallbackIterator>
  Public:
    callback: function (x) 
    clone: function (deep = FALSE) 
    initialize: function (x, callback = identity) 
    is_complete: active binding
    length: active binding
    move_cursor: function () 
    nextElem: function () 
    x: GenericIterator, iterator, R6

Now, we need to create a _vocabulary_ dataframe and run some filtering on it.  `text2vec` will use this dataframe to convert the token iterator into a matrix in a repeatable fashion.

In [8]:
vocab <- create_vocabulary(tokens)
head(vocab, 10)

,term,term_count,doc_count
,<chr>,<int>,<int>
1,aaaand,1,1
2,aand,1,1
3,aback,1,1
4,abarth,1,1
5,abdl,1,1
6,abena,1,1
7,abf,1,1
8,abi,1,1
9,abiut,1,1


Let's filter out super rare and super common words.  Anything we filter out will be ignored when `text2vec` creates the sparse matrix in a bit here.

In [9]:
vocab <- (
    vocab
    %>% filter(doc_count > 10)
    %>% filter(doc_count < (sum(vocab$doc_count) / 2))
)

Now we create a _vectorizer_ based on the vocab table--this is what the other functions will use to create a document-term matrix with consistent columns.

In [10]:
# vectorizer
vectorizer <- vocab_vectorizer(vocab)
# create a document-term matrix
co_occurrence_matrix <- create_dtm(tokens, vectorizer)
co_occurrence_matrix[1:10, 1:10]

as(<dgTMatrix>, "dgCMatrix") is deprecated since Matrix 1.5-0; do as(., "CsparseMatrix") instead

  [[ suppressing 10 column names 'abc', 'absent', 'ai' ... ]]



10 x 10 sparse Matrix of class "dgCMatrix"
                      
1  . . . . . . . . . .
2  . . . . . . . . . .
3  . . . . . . . . . .
4  . . . . . . . . . .
5  . . . . . . . . . .
6  . . . . . . . . . .
7  . . . . . . . . . .
8  . . . . . . . . . .
9  . . . . . . . . . .
10 . . . . . . . . . .

Let's combine that all into a single group of functions:

In [11]:
preprocess <- function(s) {
    return (
        s$review_body
        %>% tolower()
        %>% gsub("[^a-z]+", " ", .)
        %>% word_tokenizer()
        %>% lapply(wordStem)
        %>% itoken()
    )
}

train_tokens <- preprocess(train)
test_tokens <- preprocess(test)

vectorizer <- (
    train_tokens
    %>% create_vocabulary()
    %>% filter(doc_count > 10)
    %>% filter(doc_count < (sum(.$doc_count) / 2))
    %>% vocab_vectorizer()
)

train_bow <- create_dtm(train_tokens, vectorizer)
test_bow <- create_dtm(test_tokens, vectorizer)

Now we can fit and evaluate the Naive Bayes model as we did with `tidytext`.

In [12]:
# Now fit the Naive Bayes model as before.
nb <- bernoulli_naive_bayes(train_bow, as.factor(train$stars))
preds <- predict(nb, newdata = test_bow)

Warning message:
"bernoulli_naive_bayes(): there are 994 empty cells leading to zero estimates. Consider Laplace smoothing."


In [13]:
print("Accuracy:")
mean(preds == test$stars)

[1] "Accuracy:"


[1] 0.381

In [14]:
print("F1 score:")
f_meas(
    data = data.frame(preds = preds, true = as.factor(test$stars)),
    preds,
    true,
    beta = 1
)

[1] "F1 score:"


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
f_meas,macro,0.3608865


Ignore the fact that the accuracy went down compared to the `tidytext` notebook; we didn't do things like stopword filtering.  We could tweak the preprocessing we did here to get comparable results if we wanted to; as it stands, the comparison is a little apples-to-oranges.